In [3]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import json
import shutil

import albumentations as A
from tensorflow.keras.layers import Conv2D,BatchNormalization,LeakyReLU,Flatten,Dense,Reshape,Dropout,GlobalMaxPool2D
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler

In [4]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg','2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg','2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg','2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg',
          '2007_000346.jpg','2007_000363.jpg','2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg','2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg','2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg','2007_000648.jpg','2007_000661.jpg',
          '2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg','2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg','2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg',
          '2007_000822.jpg','2007_000830.jpg','2007_000836.jpg','2007_000837.jpg','2007_000847.jpg']

In [5]:
train_images=r"C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\pascal-voc-2012\VOC2012\JPEGImages"
train_maps=r"C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\pascal-voc-2012\VOC2012\Annotations"

val_images=r"C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\pascal-voc-2012\VOC2012\ValJPEGImages"
val_maps=r"C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\pascal-voc-2012\VOC2012\ValAnnotations"

classes=["aeroplane","bicycle","bird","boat","bottle","bus","car","cat","chair","cow","diningtable","dog","horse","motorbike","person","pottedplant","sheep","sofa","train","tvmonitor"]
B=2
N_CLASSES=len(classes)
OBJ_IND=N_CLASSES+5*B
H,W=224,224
SPLIT_SIZE=H//32
N_EPOCHS=135
BATCH_SIZE=32

In [11]:
for name in val_list:
    shutil.move(train_images+"/" +name,val_images+'/'+name)
    shutil.move(train_maps+'/'+name[:-3]+"xml",val_maps+'/'+name[:-3]+"xml")

    

In [6]:
def preprocess_xml(filename):
    
    
    tree = ET.parse(filename)
    root=tree.getroot()
    size_tree=root.find('size')
    width=float(size_tree.find('width').text)
    height=float(size_tree.find('height').text)

    bounding_boxes=[]

    for object_tree in root.findall('object'):
        
        class_name= object_tree.find('name').text
        class_id=classes.index(class_name)
        xmin=float(object_tree.find('bndbox').find('xmin').text)
        ymin=float(object_tree.find('bndbox').find('ymin').text)
        xmax=float(object_tree.find('bndbox').find('xmax').text)
        ymax=float(object_tree.find('bndbox').find('ymax').text)

        center_x=(xmax-xmin)/(width*2)
        center_y=(ymax-ymin)/(height*2)
        w=(xmax-xmin)/(width)
        h=(ymax-ymin)/(height)

        bounding_boxes.append([center_x,center_y,w,h,class_id])

    return tf.convert_to_tensor(bounding_boxes)
        
    

In [7]:
preprocess_xml(train_maps+"/2007_000862.xml")


<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[1.3000000e-02, 2.6666667e-02, 2.6000001e-02, 5.3333335e-02,
        1.4000000e+01],
       [5.0000001e-02, 2.4000000e-02, 1.0000000e-01, 4.8000000e-02,
        3.0000000e+00]], dtype=float32)>

In [8]:
def generate_output(bounding_boxes):
    output_label=np.zeros((7,7,25))

    for b in range(len(bounding_boxes)):
        grid_x=bounding_boxes[b][0]*7
        grid_y=bounding_boxes[b][1]*7
        i=int(grid_x)
        j=int(grid_y)

        output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[b][2],bounding_boxes[b][3]]
        output_label[i,j,5+int(bounding_boxes[b][4])]=1

    return tf.convert_to_tensor(output_label,tf.float32)

In [9]:
generate_output(preprocess_xml(train_maps+"/2007_000862.xml"),)

<tf.Tensor: shape=(7, 7, 25), dtype=float32, numpy=
array([[[1.   , 0.35 , 0.168, ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ]],

       [[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ]],

       [[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0

In [10]:
im_paths=[]
xml_paths=[]

val_im_paths=[]
val_xml_paths=[]

for i in os.listdir(train_maps):
    im_paths.append(train_images+"/"+i[:-3]+'jpg')
    xml_paths.append(train_maps+'/'+i)

for i in os.listdir(val_maps):
    val_im_paths.append(val_images+"/"+i[:-3]+'jpg')
    val_xml_paths.append(val_maps+'/'+i)


print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

17061 17061
64 64


In [11]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))




In [12]:
tf.config.run_functions_eagerly(True)

def get_imbboxes(im_path,xml_path):

  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img,size=(H,W)),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess_xml,inp=[xml_path],Tout=tf.float32)

  return img,bboxes

In [13]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

c:\users\mehmet\desktop\python\computer vision\yolo_object_detection\myenv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [14]:
for i,j in train_dataset.take(1):
  print(i.shape,j)

(224, 224, 3) tf.Tensor(
[[1.3000000e-02 2.6666667e-02 2.6000001e-02 5.3333335e-02 1.4000000e+01]
 [5.0000001e-02 2.4000000e-02 1.0000000e-01 4.8000000e-02 3.0000000e+00]], shape=(2, 5), dtype=float32)


In [15]:
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),

], bbox_params=A.BboxParams(format='yolo', ))

In [16]:
def aug_albument(image,bboxes):
    augmented=transforms(image=image,bboxes=bboxes)
    return [tf.convert_to_tensor(augmented['image'],dtype=tf.float32),
            tf.convert_to_tensor(augmented['bboxes'],dtype=tf.float32)]

In [17]:
def process_data (image,bboxes):
    aug=tf.numpy_function(func=aug_albument,inp=[image,bboxes],Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [18]:
train_dataset=train_dataset.map(process_data)

In [19]:
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [20]:
def preprocess(img,y):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [21]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [22]:
train_dataset=(
    train_dataset.
    batch(BATCH_SIZE,drop_remainder=True).
    
    prefetch(tf.data.AUTOTUNE)
)
val_dataset=(
    val_dataset.
    batch(BATCH_SIZE,drop_remainder=True).
    prefetch(tf.data.AUTOTUNE)
)

In [23]:
print(len(train_dataset))

533


In [24]:
for i,j in train_dataset.take(1):
  print(i.shape,j)

(32, 224, 224, 3) tf.Tensor(
[[[[1.         0.29000002 0.17028055 ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.  

In [25]:
NUM_FILTERS=512
OUTPUT_DIM=30

#base_model =tf.keras.applications.resnet50.ResNet50(
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)

base_model.trainable=False

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [26]:
model=tf.keras.Sequential([

    base_model,

    Conv2D(NUM_FILTERS,kernel_size=(3, 3), padding="same",kernel_initializer="he_normal",),
    BatchNormalization(),
    LeakyReLU(negative_slope=0.1),

    Conv2D(NUM_FILTERS,kernel_size=(3, 3), padding="same",kernel_initializer="he_normal",),
    BatchNormalization(),
    LeakyReLU(negative_slope=0.1),

    Conv2D(NUM_FILTERS,kernel_size=(3, 3), padding="same",kernel_initializer="he_normal",),
    BatchNormalization(),
    LeakyReLU(negative_slope=0.1),

    Conv2D(NUM_FILTERS,kernel_size=(3, 3), padding="same",kernel_initializer="he_normal",),
    LeakyReLU(negative_slope=0.1),

    Flatten(),

    Dense(NUM_FILTERS,kernel_initializer="he_normal",),
    BatchNormalization(),
    LeakyReLU(negative_slope=0.1),

    Dropout(0.5),
    
    Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM,activation='sigmoid'),


    Reshape((SPLIT_SIZE,SPLIT_SIZE,OUTPUT_DIM)),

])

model.build(input_shape=(None, 224, 224, 3))
model.summary()
#example_input = np.random.random((32, H, W, 3))

#Modelden geçirme
#output = model(example_input)

# Her katmanın çıktısını kontrol et
#for layer in model.layers:
#    layer_output = layer(example_input)
#    print(f"{layer.name} output shape: {layer_output.shape}")
#    example_input = layer_output  # Sonuçları bir sonraki katman için girdi olarak kullan

# Son katman çıktısını kontrol et
#print(f"Final output shape: {output.shape}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb1 (Functional)          │ (None, 7, 7, 1280)          │       6,575,239 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 7, 7, 512)           │       5,898,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1470)                │         754,110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 7, 7, 30)            │               0 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 33,161,285 (126.50 MB)

 Trainable params: 26,581,950 (101.40 MB)

 Non-trainable params: 6,579,335 (25.10 MB)

In [27]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [28]:
def difference(x,y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.float32)
    return tf.reduce_sum(tf.square(y-x))

In [29]:
def yolo_loss(y_true,y_pred):

  #print(y_true.shape)
  #print(y_pred.shape)
  target =y_true[...,0]

  ## Object loss
  y_pred_extract=tf.gather_nd(y_pred,tf.where(target[:]==1))
  y_target_extract =tf.gather_nd(y_true, tf.where(target[:]==1))

  #print(tf.where(target[:]==1))
  #print(y_pred_extract)
  #print(y_target_extract)


  rescaler=tf.where(target[:]==1)*32
  #tf.print('rescaler',rescaler)
  upscaler_1=tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2],dtype=tf.int64)],axis=-1)
  #tf.print('upscaler_1',upscaler_1)

  target_upscaler_2=tf.repeat([[32.,32.,224.,224.]],repeats=[len(rescaler)],axis=0)*tf.cast(y_target_extract[...,1:5],dtype=tf.float32)
  #tf.print('target_upscaler_2',target_upscaler_2)

  pred_1_upscaler_2=tf.repeat([[32.,32.,224.,224.]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,1:5],dtype=tf.float32)
  #tf.print('pred_1_upscaler_2',pred_1_upscaler_2)

  pred_2_upscaler_2=tf.repeat([[32.,32.,224.,224.]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,6:10],dtype=tf.float32)
  #tf.print('pred_2_upscaler_2',pred_2_upscaler_2)

  target_orig=tf.cast(upscaler_1,dtype=tf.float32)+target_upscaler_2
  pred_1_orig=tf.cast(upscaler_1,dtype=tf.float32)+pred_1_upscaler_2
  pred_2_orig=tf.cast(upscaler_1,dtype=tf.float32)+pred_2_upscaler_2

  mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_2_orig),
                                         compute_iou(target_orig,pred_1_orig)),dtype=tf.int32)

  y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                        tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0))

  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

  object_loss = difference(tf.cast(obj_pred,dtype =tf.float32)
                            ,tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

  ####################### For No object
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==0))
  y_target_extract = tf.zeros(len(y_pred_extract))

  no_object_loss_1 = difference(tf.cast(y_pred_extract[...,0],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss_2 = difference(tf.cast(y_pred_extract[...,5],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss = no_object_loss_1+no_object_loss_2

  ######################## For OBject class loss
  y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
  class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))
  #tf.print("y_pred_extract shape:", tf.shape(y_pred_extract))
  #tf.print("class_extract shape:", tf.shape(class_extract))


  class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32)
                                ,tf.cast(class_extract,dtype=tf.float32))

  ######################### For object bounding box loss
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==1))
  centre_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)
  centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  centre_target = tf.gather_nd(y_true[...,1:3], tf.where(target[:]==1))

  centre_loss = difference(centre_pred,centre_target)

  size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)

  size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  size_target = tf.gather_nd(y_true[...,3:5], tf.where(target[:]==1))

  size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))
  box_loss = centre_loss+size_loss

  lambda_coord = 5.0
  lambda_no_obj = 0.5

  loss = object_loss + (lambda_no_obj*no_object_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
  return loss

In [30]:
 y_true=generate_output([[0.210784,0.616422,0.127451,0.232843,2]])
 #y_true=generate_output([[0.509804,0.411765,0.107843,0.245098,3],
  #                       [0.210784,0.616422,0.127451,0.232843,2]])
 y_true=np.expand_dims(y_true,axis=0)
 y_pred=np.random.normal(size = (1,7,7,N_CLASSES+5*B))

 y_pred[0][1][4] = [0.9,0.47,0.31,0.12,0.23,   1.0,0.2,0.6,0.1,0.95,    0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35,0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35,0.1,0.5,0.9,0.35]
 y_pred[0][3][2] = [0.3,0.01,0.08,0.11,0.54,   0.98,0.56,0.88,0.1,0.24, 0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3,0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3,0.01,0.05,0.99,0.3]

 yolo_loss(y_true,y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=61.51118469238281>

In [31]:
checkpoint_filepath=r'C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\checkpoints\yolo_resnet_50.weights.h5'

callback= tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [32]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch >40 and epoch < 80:
    return 5e-4
  else:
    return 1e-4

In [33]:
lr_callback=tf.keras.callbacks.LearningRateScheduler(scheduler)

In [34]:
model.compile(
    loss=yolo_loss,
    optimizer=Adam(1e-3),
)

In [ ]:
history=model.fit(
    train_dataset,
    validation_data=val_dataset,
    verbose=1,
    epochs=90,
    callbacks=[lr_callback,callback]
    
)

Epoch 1/90


c:\users\mehmet\desktop\python\computer vision\yolo_object_detection\myenv\lib\site-packages\albumentations\core\bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)


259/533 ━━━━━━━━━━━━━━━━━━━━ 30:41 7s/step - loss: 240.7540

In [ ]:
model.save(r"C:\Users\mehmet\Desktop\Python\Computer Vision\yolo_object_detection\Pascal_VOC_Yolo\dataset\models\my_model.keras")


In [ ]:
plt.figure(figsize=(14, 6))



# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], color='forestgreen', linewidth=2, marker='o')
plt.plot(history.history['val_loss'], color='crimson', linewidth=2, marker='o')
plt.title('Model Loss', fontsize=16, fontweight='bold')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(['Train', 'Validation'], fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)


plt.tight_layout()
plt.show()


print("Training history plotted.")